In [77]:
!pip install rdflib
!pip install urllib3
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from google.colab import drive

drive.mount('/content/gdrive')
ontology_url = 'gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/tanks.owl'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [33]:
from rdflib import Graph

# Create a Graph
tanks_ontology_graph = Graph()

# Parse in an RDF file
tanks_ontology_graph.parse(ontology_url)

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in tanks_ontology_graph:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in tanks_ontology_graph:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"tanks_ontology_graph has {len(tanks_ontology_graph)} statements.")
# Prints: Graph g has 86 statements.

# Print out the entire Graph in the RDF Turtle format
# print(g.serialize(format="turtle"))

tanks_ontology_graph has 1023 statements.


# Network scripts
In below section I implemented list of methods that are used for get data from Internet

In [79]:
import urllib3
import json
from bs4 import BeautifulSoup as bs

urllib3.disable_warnings()

http = urllib3.PoolManager()
application_id = 'a12842cff8d85bdbdb9e779cca01397b'

# Script for obtain all 10 tier tanks info
def get_all_tanks_json():
  r = http.request('GET', f"https://api.tanki.su/wot/encyclopedia/vehicles/?application_id={application_id}&tier=10&fields=is_wheeled%2Ctag%2Ctank_id%2Ctype%2Cdescription%2Cshort_name%2Cnation%2Ctier%2Cname%2Cdefault_profile.hp%2Cdefault_profile.speed_forward%2Cdefault_profile.speed_backward%2Cdefault_profile.engine%2Cdefault_profile.ammo%2Cdefault_profile.modules%2Cdefault_profile.gun%2Cdefault_profile.turret%2Cdefault_profile.armor%2Cdefault_profile.rapid%2Cdefault_profile.siege")
  data = json.loads(r.data)
  res = data['data']
  return res

# Script for obtain player info
def get_user_info_by_nickname(nickname):
  ## get id by nickname
  r = http.request('GET', f"https://api.tanki.su/wot/account/list/?application_id={application_id}&search={nickname}")
  response = json.loads(r.data)
  data = response['data']
  player_id = None
  for player in data:
    if player['nickname'] == nickname:
      player_id = player['account_id']
  
  ## get info by id
  player_info = None
  if player_id != None:
    r = http.request('GET', f"https://api.tanki.su/wot/account/info/?application_id={application_id}&account_id={player_id}&fields=account_id%2Cglobal_rating%2Cclan_id%2Cstatistics.all%2Cstatistics.trees_cut%2Cnickname")
    data = json.loads(r.data)
    player_info = data['data'][f'{player_id}']

  return player_info

# Script for obtain stats on each tank by player
def get_player_stats_on_tank_by_id(player_id):
  r = http.request('GET', f"https://api.tanki.su/wot/tanks/stats/?account_id={player_id}&application_id={application_id}&in_garage=1&tank_id=20225%2C3473%2C7169%2C6193%2C15425%2C6145%2C5281%2C13569%2C32001%2C8705%2C2417%2C15953%2C14609%2C16897%2C16913%2C13857%2C3649%2C50849%2C6209%2C34049%2C7249%2C14113%2C9233%2C9297%2C52081%2C5265%2C5425%2C61697%2C4145%2C13825%2C20993%2C14881%2C13905%2C62513%2C4737%2C2721%2C22017%2C17217%2C55841%2C16961%2C58369%2C19489%2C6225%2C21537%2C2977%2C21761%2C8497%2C17729%2C5505%2C12369%2C50689%2C15905%2C13889%2C3681%2C63537%2C12049%2C57937%2C60977%2C6929%2C62257%2C35361%2C19217%2C3937%2C22273%2C11841%2C19473%2C10785%2C9489%2C12305%2C58961%2C8481%2C15697%2C2433%2C2929%2C19985%2C58641%2C5937%2C19969%2C46849%2C19201%2C15617%2C13089%2C19009&fields=account_id%2Cmax_xp%2Call%2Cmax_frags%2Cfrags%2Cmark_of_mastery%2Ctank_id")
  data = json.loads(r.data)
  res = data['data'][f'{player_id}']
  return res

# Script for obtain clan stats
def get_clan_info_by_tag(clan_tag):
  ## get id by tag
  r = http.request('GET', f"https://api.tanki.su/wot/clans/list/?application_id={application_id}&search={clan_tag}")
  response = json.loads(r.data)
  data = response['data']
  clan_id = None
  for clan in data:
    if clan['tag'] == clan_tag:
      clan_id = clan['clan_id']
  
  ## get info by id
  clan_info = None
  if clan_id != None:
    r = http.request('GET', f"https://api.tanki.su/wot/clans/info/?application_id={application_id}&clan_id={clan_id}&fields=members_count%2Cdescription%2Ccreator_name%2Cclan_id%2Cleader_name%2Ccreator_id%2Ctag%2Cmembers.account_id%2Cmembers.account_name%2Cold_name%2Cleader_id%2Cmotto%2Cold_tag%2Cname")
    data = json.loads(r.data)
    clan_info = data['data'][f'{clan_id}']

  return clan_info

# Script to obtain battle info
# def get_battle_info(battle_id):
#   r = http.request('GET', f"http://tankireplays.lesta.ru/site/{battle_id}#stats")
#   soup = bs(r.data, 'html.parser')

#   ## get battle status (win/lose)
#   battle_status = soup.find('div', class_='replay-stats__title').text

#   ## get teams and result of each player
#   ally_team_results = []
#   enemy_team_results = []

#   r = http.request('GET', f"http://tankireplays.lesta.ru/site/{battle_id}#teams")
#   soup = bs(r.data, 'html.parser')
#   ally_team_html = soup.find('div', class_='replay-teams__team ng-scope replay-teams__team--ally')
#   enemy_team_html = soup.find('div', class_='replay-teams__team ng-scope replay-teams__team--enemy')

#   print(soup.prettify())
#   print(ally_team_html)
#   print(enemy_team_html)

#   battle = {
#       'id': battle_id,
#       'status': battle_status,
#       'ally_team_results': ally_team_results,
#       'enemy_team_results': enemy_team_results
#   }

#   return battle

In [35]:
all_tanks = get_all_tanks_json()
print(f"number of tanks: {len(all_tanks)}")

player_json = get_user_info_by_nickname('preachooda')
print(f"{player_json['nickname']}'s id: {player_json['account_id']}")

player_stats_on_tanks_json = get_player_stats_on_tank_by_id(player_json['account_id'])
print(f"number of tanks that owned by {player_json['nickname']}: {len(player_stats_on_tanks_json)}")

clan_json = get_clan_info_by_tag('NE6P0')
print(f"number of {clan_json['name']} clan members: {len(clan_json['members'])}")

number of tanks: 83
preachooda's id: 8438923
number of tanks that owned by preachooda: 31
number of Ne6p0 clan members: 91


In [36]:
tank = None
for id, info in all_tanks.items():
  tank = info
  break
  # if info['nation'] == 'czech':
  #   print(f"tank id = {id}; tank name = {info['short_name']}")

In [37]:
# Show tanks info
# tank

In [38]:
# Show common player stats
# player_json

In [39]:
# Show one entity of stats on concrete tank for player
# player_stats_on_tanks_json[0]

In [40]:
# Show clan info
# clan_json